In [1]:
from jinja2 import Template
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,get_response_synthesizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings,PromptTemplate
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.retrievers import VectorIndexRetriever
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI

/Users/hyjiang/miniforge3/envs/py311-tra/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = OpenAI(api_key="860f7bb0-8efc-4b48-8498-88ec0ef8a60e",base_url="https://api-inference.modelscope.cn/v1/")

In [3]:
# 格式化输出

class choice(BaseModel):
    """正确的选项"""

    correct_chice: str

In [4]:
prompt_choice='''
{{question}}
{{choice}}
'''
choice_template = Template(prompt_choice)

prompt_jianda='''
{{question}}
'''
jianda_template = Template(prompt_jianda)


extract_answer = '''
你是一个答案提取助手，请直接从我给出的包含答案选项和解析的文本中提取出正确答案选项,只返回选项对应的字母，不返回任何其他内容。

文本陈述:
{{chenshu}}
'''

jian_template = Template(extract_answer)

In [5]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3",)

In [8]:
documents = SimpleDirectoryReader(
    "/Users/hyjiang/Downloads/tmp_downloads/2025xib_厦门国际银行数创金融杯/初赛/制度文档demo",
    recursive=True
).load_data(show_progress=True)

Loading files: 100%|██████████| 2/2 [00:00<00:00, 38.36it/s]


In [9]:
Settings.llm = OpenAILike(
    model="Qwen/Qwen3-8B",
    api_base='https://api-inference.modelscope.cn/v1/',
    api_key='860f7bb0-8efc-4b48-8498-88ec0ef8a60e',
    is_chat_model=True,
    temperature=0.6,
    top_p = 0.95,
    top_k = 20,
    additional_kwargs={
        "extra_body": {"enable_thinking": False},
    },
)

index = VectorStoreIndex.from_documents(documents)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

In [10]:
response_synthesizer = get_response_synthesizer(
    
        response_mode="tree_summarize"
        # output_cls=choice
)

In [11]:
# 修改text_qa_template
new_choice_tmpl_str = (    "你是一个兼容监管制度知识专家。下面是上下文信息。\n"    "{context_str}\n"    "你将会再查询接受到一个金融监管制度方面不定项选择题和多个选项。请根据上下文信息和非先验知识, 在给出的选项中选出正确的答案（可能是单个或多个选项)"    "查询：{query_str}\n"    "答案:")
new_choice_tmpl = PromptTemplate(new_choice_tmpl_str)

new_jianda_tmpl_str = (    "你是一个兼容监管制度知识专家。下面是上下文信息。\n"    "{context_str}\n"    "你将会再查询接受到一个金融监管制度方面简答题。请根据上下文信息和非先验知识, 对问题做出正确的解答"    "查询：{query_str}\n"    "答案:")
new_jianda_tmpl = PromptTemplate(new_jianda_tmpl_str)

In [12]:
# query_engine = index.as_query_engine(
#         response_mode="tree_summarize",
#         output_cls=choice
# )

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.6)],
)

In [14]:
data = pd.read_json('/Users/hyjiang/Downloads/tmp_downloads/2025xib_厦门国际银行数创金融杯/初赛/数据集demo/demo_train.json',lines=True)
results = {
    "id":[],
    "answer":[]
}
for idx,row in data.iterrows():
    print("=====================================")
    if row['category']=='选择题':
        query_engine.update_prompts(
            {"response_synthesizer:summary_template": new_choice_tmpl}
        )
        prompt = choice_template.render(question=row['question'], choice = row['content'])
        response = query_engine.query(prompt)
        print(response)
        chenshu = jian_template.render(chenshu=response)
        message = [
        # {'role':"system","content":sys},
        {"role": "user", "content": chenshu}]
        result = client.chat.completions.create(messages=message,
                                                model="Qwen/Qwen3-8B",
                                                temperature=0.1,
                                                top_p=0.8,
                                                max_tokens=1000,
                                                extra_body={
                                                    "repetition_penalty": 1.05,
                                                    "enable_thinking": False
                                                },)
        print(result.choices[0].message.content)
        result = [result.choices[0].message.content.strip()]
        results["id"].append(row["id"])
        results["answer"].append(result)
    else:
        print("============================================")
        query_engine.update_prompts(
            {"response_synthesizer:summary_template":new_jianda_tmpl}
        )
        prompt = jianda_template.render(question=row['question'])
        response = query_engine.query(prompt)
        print(response)
        results["id"].append(row["id"])
        results["answer"].append(result)

根据提供的上下文信息，该文件是中国银行业监督管理委员会（现为中国银行保险监督管理委员会）于2007年发布的《关于加强商业性房地产信贷管理的通知》（银发〔2007〕359号），其内容主要涉及房地产信贷管理的若干规定，包括住房消费贷款、商业用房购房贷款、房地产开发贷款、土地储备贷款以及房地产信贷征信管理等方面。

然而，**该文件并未提及商业银行的流动性覆盖率（LCR）相关监管要求**。流动性覆盖率是巴塞尔协议III中引入的重要监管指标，用于衡量银行在短期内（30天内）应对流动性需求的能力，通常由各国金融监管机构根据国际标准制定具体要求。

因此，在给定的上下文中，并没有关于“商业银行的流动性覆盖率最低监管标准”的信息。

### 正确答案：  
**D. 根据资产规模动态调整**

**理由**：由于上下文未提供具体的流动性覆盖率数值，且该通知仅针对房地产信贷管理，因此无法从文中直接得出具体数值。而选项D“根据资产规模动态调整”更符合实际监管实践中对流动性覆盖率的处理方式，即不同规模的银行可能有不同的监管标准。
D
根据提供的上下文信息，内容主要围绕**商业性房地产信贷管理**，包括住房消费贷款、商业用房购房贷款、房地产开发贷款、土地储备贷款等领域的监管要求。文中并未提及任何关于**绿色信贷**或**绿色项目支持**的内容。

因此，从给定的上下文信息中无法直接推断出银行业发展绿色信贷时应优先支持的项目类型。

但根据**金融监管的一般原则**和国家政策导向（如“双碳”目标、生态文明建设等），绿色信贷通常优先支持**节能环保、可持续发展**类项目。

在选项中：

- **A. 落后产能扩建** —— 不符合绿色信贷方向，可能增加污染。
- **B. 符合标准的节能环保项目** —— 符合绿色信贷支持方向。
- **C. 高污染工业设施** —— 明显不符合绿色信贷支持方向。
- **D. 房地产开发** —— 从上下文看，房地产开发贷款有严格限制，且不一定是绿色信贷优先支持对象。

### 正确答案：**B. 符合标准的节能环保项目**
B
Empty Response
无
Empty Response
无
Empty Response
无
Empty Response
无
Empty Response
无
Empty Response
Empty Response